In [31]:
!pip install lxml
!pip install selenium
!pip install cssselect
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

In [32]:
import pandas as pd
import numpy as np

from lxml import html
from lxml.cssselect import CSSSelector

# Create the web driver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [33]:
# Load the site on the web driver
url = "https://axie.zone/leaderboard"
driver.get(url)
driver.implicitly_wait(10)

# List to store links pointing to the player pages
players = []

try:
  # Find the anchor tags in the leaderboard table
  table_body_selector = ".leaderboard_table tbody"
  table_body = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, table_body_selector)))
  
  table_html = table_body[0].get_attribute('innerHTML')
  table_tree = html.fromstring(table_html)

  table_rows = table_tree.cssselect("tr")

  # Extract the href from the anchors found in the table
  for row in table_rows:
    players.append({
        "url": "https://axie.zone/" + row.cssselect("a")[0].get("href"),
        "winrate": int(row.cssselect("td:nth-child(6)")[0].text.replace("%", ""))
    })

  print("Retrieved", len(players), "URLs")
except TimeoutException:
  print("Element not properly loaded")

print(players)

Retrieved 100 URLs
[{'url': 'https://axie.zone//profile?ron_addr=0x6f08189713d7c88b6f69b671c9ac5f0081cc9ba5', 'winrate': 54}, {'url': 'https://axie.zone//profile?ron_addr=0x7f7cdcb632d1641378086ce89ab4190b61677805', 'winrate': 54}, {'url': 'https://axie.zone//profile?ron_addr=0x833ce13db4f49cae55833b8671eddfe1f9a0a2db', 'winrate': 56}, {'url': 'https://axie.zone//profile?ron_addr=0xdabb232cb403ba23c44c8d593851c47ad74be52f', 'winrate': 49}, {'url': 'https://axie.zone//profile?ron_addr=0xfdaa118b4c3ca2ea10bdc89becda775c081cb7d6', 'winrate': 54}, {'url': 'https://axie.zone//profile?ron_addr=0xc5bf891f862dcb690a6d7a5a935f0df6bd6d41ab', 'winrate': 58}, {'url': 'https://axie.zone//profile?ron_addr=0xebb389d9593b86a964433f16f04cb408df67ad82', 'winrate': 55}, {'url': 'https://axie.zone//profile?ron_addr=0x9fc41da747be454189ed5b2c81755839b6607e80', 'winrate': 55}, {'url': 'https://axie.zone//profile?ron_addr=0xa56a0aae7d6da05d62be9638918e65a18702764c', 'winrate': 53}, {'url': 'https://axie.zone

In [34]:
import re

# Used to extract the Axie parts from the user page
def get_parts(players):
  # List to store the scraped data
  axie_data = []

  # Counter that will be used to track the progress of the function
  progress = 0

  # Regex pattern to parse the parts
  p = re.compile('(.*): (.*) (\[.*\])')

  # Iterate through each of the player in the leaderboards
  for player in players:
    progress += 1
    try:
      # Load the player page on the driver
      driver.get(player["url"])
      driver.implicitly_wait(10)

      # Wait till the Axie info container loads
      _ = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#last_used_team_container > div")))

      # Get the element that wraps the last used team container
      # This can be refactored to include the most used team of the player as well
      axie_container = WebDriverWait(driver, 0).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#last_used_team_container")))

      # Convert the selenium webelement into a LXML tree
      axie_html = axie_container[0].get_attribute('innerHTML')
      axie_tree = html.fromstring(axie_html)

      # Get the Axie containers
      info_containers = axie_tree.cssselect(".search_result_wrapper")

      for info_container in info_containers:
        axie_parts = {}

        # Get the address of the owner
        axie_parts["User_Addr"] = player["url"].replace("https://axie.zone/profile?ron_addr=", "")

        # Get the Axie class
        axie_parts["Class"] = info_container.cssselect(".search_result")[0].get('class').replace("search_result ", "").capitalize()

        # Scrapes the parts from the Axie container
        parts = info_container.cssselect(".purity_part > div")

        for part in parts:
          m = p.match(part.get('title'))
          axie_parts[m.group(1)] = m.group(2)

        # Add winrate
        axie_parts["Winrate"] = player["winrate"]

        # Add the scraped parts to the main list of data
        axie_data.append(axie_parts)

      # To keep track and now that our script is working, we would print a message
      # containing a summary of the scraping's progress
      print("(", progress, "/ 100 ) Retrieved")
    except Exception as e:
      # We might also encounter an error where our collection script fails,
      # in the rare case where this would happen we would need to manually
      # revisit what went wrong and fix it.
      print("(", progress, "/ 100 ) Error occured while retrieving", player["url"])
    break

  return axie_data
  
# Start the scrapping process
res = get_parts(players)

( 1 / 100 ) Retrieved


In [35]:
# Convert the python list into a dataframe
df_res = pd.DataFrame(res)
df_res.head()

,User_Addr,Class,Eyes,Ears,Back,Mouth,Horn,Tail,Winrate
0,https://axie.zone//profile?ron_addr=0x6f081897...,Aquatic,Clear,Leafy,Goldfish,Piranha,Cuckoo,Koi,54
1,https://axie.zone//profile?ron_addr=0x6f081897...,Beast,Little Owl,Risky Bird,Ronin,Nut Cracker,Imp,Nut Cracker,54
2,https://axie.zone//profile?ron_addr=0x6f081897...,Plant,Cucumber Slice,Lotus,Pumpkin,Serious,Leaf Bug,Cattail,54


In [36]:
# Export the dataframe into a csv file
df_res.to_csv("leaderboard.csv", index=False)